# Домашнее задание к лекции "Статистика. Практика"

## Задание 1

Вернемся к [набору данных о видеоиграх](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/vgsales.csv).

Ответьте на следующие вопросы:

1) Как критики относятся к спортивным играм?  
2) Критикам нравятся больше игры на PC или на PS4?  
3) Критикам больше нравятся стрелялки или стратегии?  

Для каждого вопроса:
- сформулируйте нулевую и альтернативную гипотезы;
- выберите пороговый уровень статистической значимости;
- опишите полученные результаты статистического теста.

In [8]:
import pandas as pd
import numpy as np
from scipy import stats as st
games = pd.read_csv('vgsales.csv', encoding='utf-8')
games_df = games.filter(items = ['Platform', 'Genre', 'Critic_Score'])

games_df = games_df.dropna()

games_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8137 entries, 0 to 16709
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Platform      8137 non-null   object 
 1   Genre         8137 non-null   object 
 2   Critic_Score  8137 non-null   float64
dtypes: float64(1), object(2)
memory usage: 254.3+ KB


In [9]:
games.head(10)

,Name,Platform,Year_of_Release,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,Wii Sports,Wii,2006.0,Sports,Nintendo,41.36,28.96,3.77,8.45,82.53,76.0,51.0,8,322.0,Nintendo,E
1,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24,NaN,NaN,NaN,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.68,12.76,3.79,3.29,35.52,82.0,73.0,8.3,709.0,Nintendo,E
3,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.61,10.93,3.28,2.95,32.77,80.0,73.0,8,192.0,Nintendo,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37,NaN,NaN,NaN,NaN,NaN,NaN
5,Tetris,GB,1989.0,Puzzle,Nintendo,23.20,2.26,4.22,0.58,30.26,NaN,NaN,NaN,NaN,NaN,NaN
6,New Super Mario Bros.,DS,2006.0,Platform,Nintendo,11.28,9.14,6.50,2.88,29.80,89.0,65.0,8.5,431.0,Nintendo,E
7,Wii Play,Wii,2006.0,Misc,Nintendo,13.96,9.18,2.93,2.84,28.92,58.0,41.0,6.6,129.0,Nintendo,E
8,New Super Mario Bros. Wii,Wii,2009.0,Platform,Nintendo,14.44,6.94,4.70,2.24,28.32,87.0,80.0,8.4,594.0,Nintendo,E
9,Duck Hunt,NES,1984.0,Shooter,Nintendo,26.93,0.63,0.28,0.47,28.31,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
games_df.head(10)

,Platform,Genre,Critic_Score
0,Wii,Sports,76.0
2,Wii,Racing,82.0
3,Wii,Sports,80.0
6,DS,Platform,89.0
7,Wii,Misc,58.0
8,Wii,Platform,87.0
11,DS,Racing,91.0
13,Wii,Sports,80.0
14,X360,Misc,61.0
15,Wii,Sports,80.0


### Как критики относятся к спортивным играм? 

H0: Средняя оценка спортивных игр <= 76  
H1: Средняя оценка спортивных игр > 76  
За alpha будем принимать 0.05

In [11]:
games_df_sports = games_df[games_df.Genre == 'Sports'].dropna()
games_df_sports.head()

,Platform,Genre,Critic_Score
0,Wii,Sports,76.0
3,Wii,Sports,80.0
13,Wii,Sports,80.0
15,Wii,Sports,80.0
77,PS4,Sports,82.0


In [32]:
alpha = 0.05
result_sports = st.ttest_1samp(games_df_sports['Critic_Score'], 70)

print(games_df_sports['Critic_Score'].mean())
print(result_sports)

if result_sports.pvalue/2 < alpha: 
    print('Отвергаем нулевую гипотезу, среднее больше 70')
else:
    print('Не отвергаем нулевую нулевую гипотезу')

71.96817420435511
Ttest_1sampResult(statistic=-0.0792393702421224, pvalue=0.9368555190248355)
Не отвергаем нулевую нулевую гипотезу


In [ ]:
# почему-то не удается обновить ни numpy ни scipy до последней версии, поэтому не использовала аргумент alternative='greater' 
# перепробовала разные способы, но пока не смогла победить, займусь вопросом на досуге

### 2. Критикам нравятся больше игры на PC или на PS4?

H0: Критикам нравятся игры на PC и на PS4 одинаково  
H1: Критики имеют разлитчное отношение к играм на PC и играм на PS4  
За alpha будем принимать 0.05

In [13]:
#сформируем две выборки с оценками критиков для каждой из платформ и удалим пустые строки
games_df_pc = games_df[games_df.Platform == 'PC'].dropna()
games_df_ps4 = games_df[games_df.Platform == 'PS4'].dropna()

In [14]:
alpha = 0.05
result = st.ttest_ind(games_df_pc['Critic_Score'], games_df_ps4['Critic_Score'])
print(result)

if (result.pvalue < alpha):
    if (result.statistic > 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся игры на РС')
    elif (result.statistic < 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся игры на PS4')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=4.544112457929973, pvalue=6.215797831285281e-06)
Отвергаем нулевую гипотезу, критикам больше нравятся игры на РС


### 3. Критикам больше нравятся стрелялки или стратегии? 

H0: Критикам одинаково нравятся стрелялки и стратегии  
H1: Критики имеют разлитчное отношение к стрелялкам и стратегиям  
За alpha будем принимать 0.05

In [15]:
#сформируем две выборки с оценками критиков для каждой категории и удалим пустые строки
games_df_shooter = games_df[games_df.Genre == 'Shooter'].dropna()
games_df_strategy = games_df[games_df.Genre == 'Strategy'].dropna()
print(games_df_shooter.Critic_Score.mean())
print(games_df_strategy.Critic_Score.mean())

70.18114406779661
72.08609271523179


In [16]:
alpha = 0.05
result = st.ttest_ind(games_df_shooter['Critic_Score'], games_df_strategy['Critic_Score'])
print(result)

if (result.pvalue < alpha):
    if (result.statistic > 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся стрелялки')
    elif (result.statistic < 0):
        print('Отвергаем нулевую гипотезу, критикам больше нравятся стратегии')
else:
    print('Не отвергаем нулевую гипотезу')

Ttest_indResult(statistic=-2.06670263948504, pvalue=0.03896871303135602)
Отвергаем нулевую гипотезу, критикам больше нравятся стратегии


In [17]:
print(games_df.Genre.unique())

['Sports' 'Racing' 'Platform' 'Misc' 'Action' 'Puzzle' 'Shooter'
 'Fighting' 'Simulation' 'Role-Playing' 'Adventure' 'Strategy']


## Задание 2

Реализуйте базовую модель логистической регрессии для классификации текстовых сообщений (используемые данные [здесь](https://github.com/obulygin/pyda_homeworks/blob/master/stat_case_study/spam.csv)) по признаку спама. Для этого:

1) Привидите весь текст к нижнему регистру;  
2) Удалите мусорные символы;  
3) Удалите стоп-слова;  
4) Привидите все слова к нормальной форме;  
5) Преобразуйте все сообщения в вектора TF-IDF. Вам поможет следующий код:  

```
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(df.Message)
names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
```

Можете поэкспериментировать с параметрами [TfidfVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html);  

6) Разделите данные на тестовые и тренировочные в соотношении 30/70, укажите `random_state=42`. Используйте [train_test_split](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.train_test_split.html);  

7) Постройте модель [логистической регрессии](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html), укажите `random_state=42`, оцените ее точность на тестовых данных;  

8) Опишите результаты при помощи [confusion_matrix](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html?highlight=confusion_matrix#sklearn.metrics.confusion_matrix);  

9) Постройте датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно (с указанием фактического и предсказанного).

In [18]:
import pandas as pd
import numpy as np
import re
spam = pd.read_csv('spam.csv', encoding='utf-8')
# приведем все слова к нижнему регистру
spam['Message'] = spam['Message'].str.lower()

spam.head(10)

,Category,Message
0,ham,"go until jurong point, crazy.. available only ..."
1,ham,ok lar... joking wif u oni...
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor... u c already then say...
4,ham,"nah i don't think he goes to usf, he lives aro..."
5,spam,freemsg hey there darling it's been 3 week's n...
6,ham,even my brother is not like to speak with me. ...
7,ham,as per your request 'melle melle (oru minnamin...
8,spam,winner!! as a valued network customer you have...
9,spam,had your mobile 11 months or more? u r entitle...


In [19]:
print(spam.Category.unique())

['ham' 'spam']


In [20]:
# уберем все служебные символы
spam['Message'] = [re.sub('[\W_]+',' ', message) for message in spam['Message']]
spam.head(10)

,Category,Message
0,ham,go until jurong point crazy available only in ...
1,ham,ok lar joking wif u oni
2,spam,free entry in 2 a wkly comp to win fa cup fina...
3,ham,u dun say so early hor u c already then say
4,ham,nah i don t think he goes to usf he lives arou...
5,spam,freemsg hey there darling it s been 3 week s n...
6,ham,even my brother is not like to speak with me t...
7,ham,as per your request melle melle oru minnaminun...
8,spam,winner as a valued network customer you have b...
9,spam,had your mobile 11 months or more u r entitled...


In [21]:
# разделим все на слова
spam['Message'] = [message.split() for message in spam['Message']]
spam.head(10)

,Category,Message
0,ham,"[go, until, jurong, point, crazy, available, o..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,"[u, dun, say, so, early, hor, u, c, already, t..."
4,ham,"[nah, i, don, t, think, he, goes, to, usf, he,..."
5,spam,"[freemsg, hey, there, darling, it, s, been, 3,..."
6,ham,"[even, my, brother, is, not, like, to, speak, ..."
7,ham,"[as, per, your, request, melle, melle, oru, mi..."
8,spam,"[winner, as, a, valued, network, customer, you..."
9,spam,"[had, your, mobile, 11, months, or, more, u, r..."


In [22]:
#удаляем стоп-слова
from nltk.corpus import stopwords
stopwords_set = set(stopwords.words('english'))
spam['Message'] = [[word for word in message if word not in stopwords_set] for message in spam['Message']]
spam.head(10)

,Category,Message
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, goes, usf, lives, around, though]"
5,spam,"[freemsg, hey, darling, 3, week, word, back, l..."
6,ham,"[even, brother, like, speak, treat, like, aids..."
7,ham,"[per, request, melle, melle, oru, minnaminungi..."
8,spam,"[winner, valued, network, customer, selected, ..."
9,spam,"[mobile, 11, months, u, r, entitled, update, l..."


In [23]:
#приведем слова к нормальной форме
from nltk.stem import WordNetLemmatizer
wordnet_lemmatizer = WordNetLemmatizer()
spam['Message'] = [[wordnet_lemmatizer.lemmatize(word) for word in message] for message in spam['Message']]
spam.head(10)

,Category,Message
0,ham,"[go, jurong, point, crazy, available, bugis, n..."
1,ham,"[ok, lar, joking, wif, u, oni]"
2,spam,"[free, entry, 2, wkly, comp, win, fa, cup, fin..."
3,ham,"[u, dun, say, early, hor, u, c, already, say]"
4,ham,"[nah, think, go, usf, life, around, though]"
5,spam,"[freemsg, hey, darling, 3, week, word, back, l..."
6,ham,"[even, brother, like, speak, treat, like, aid,..."
7,ham,"[per, request, melle, melle, oru, minnaminungi..."
8,spam,"[winner, valued, network, customer, selected, ..."
9,spam,"[mobile, 11, month, u, r, entitled, update, la..."


In [24]:
#обединим слова в строки
spam['Message'] = [" ".join(message) for message in spam['Message']]
spam.head(10)

,Category,Message
0,ham,go jurong point crazy available bugis n great ...
1,ham,ok lar joking wif u oni
2,spam,free entry 2 wkly comp win fa cup final tkts 2...
3,ham,u dun say early hor u c already say
4,ham,nah think go usf life around though
5,spam,freemsg hey darling 3 week word back like fun ...
6,ham,even brother like speak treat like aid patent
7,ham,per request melle melle oru minnaminunginte nu...
8,spam,winner valued network customer selected receiv...
9,spam,mobile 11 month u r entitled update latest col...


In [25]:
#преобразуем все сообщения в вектора TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
tfidf_matrix = tfidf.fit_transform(spam['Message'])

names = tfidf.get_feature_names()
tfidf_matrix = pd.DataFrame(tfidf_matrix.toarray(), columns=names)
tfidf_matrix.head(10)

,00,000,000pes,008704050406,0089,0121,01223585236,01223585334,0125698789,02,...,zhong,zindgi,zoe,zogtorius,zoom,zouk,zyada,èn,ú1,〨ud
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
# разделим данные на тестовые и тренировочные в соотношении 30/70

spam['spam/not_spam'] = (spam['Category'] == 'spam')*1
spam.head(10)

,Category,Message,spam/not_spam
0,ham,go jurong point crazy available bugis n great ...,0
1,ham,ok lar joking wif u oni,0
2,spam,free entry 2 wkly comp win fa cup final tkts 2...,1
3,ham,u dun say early hor u c already say,0
4,ham,nah think go usf life around though,0
5,spam,freemsg hey darling 3 week word back like fun ...,1
6,ham,even brother like speak treat like aid patent,0
7,ham,per request melle melle oru minnaminunginte nu...,0
8,spam,winner valued network customer selected receiv...,1
9,spam,mobile 11 month u r entitled update latest col...,1


In [27]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(tfidf_matrix, spam['spam/not_spam'], test_size=0.30, random_state=42)

In [28]:
# построим модель логистической регрессии

from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import accuracy_score

lda = LinearDiscriminantAnalysis()
lda.fit(X_train, y_train)
accuracy_score(y_test, lda.predict(X_test))

0.9677033492822966

Получили точность 96.8%

In [29]:
# описываем результаты при помощи confusion_matrix;

from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, lda.predict(X_test))



array([[1445,    3],
       [  51,  173]], dtype=int64)

Верно предсказано:
    - 1445 сообщений spam
    - 173 сообщений not spam
Неверно предсказано:
    - 51 сообщений spam
    - 3 сообщений not spam    

    
Иначе:
1445 сообщений верно отправились в спам
171 обычное сообщение верно предсказано
51 обычное сообщение отправились в спам
3 сообщения спама остались в общей папке

In [30]:
# строим датафрейм, который будет содержать все исходные тексты сообщений, классифицированные неправильно 
#(с указанием фактического и предсказанного)


spam_false = pd.concat([spam[spam.index.isin(y_test.index)].reset_index(), 
                    pd.Series(lda.predict(X_test),
                              np.arange(len(lda.predict(X_test))),
                              name = 'predict')],
                    axis=1)
spam_false = spam_false[spam_false['spam/not_spam'] != spam_false['predict']]
spam_false

,index,Category,Message,spam/not_spam,predict
0,8,spam,winner valued network customer selected receiv...,1,0
1,12,spam,urgent 1 week free membership 100 000 prize ja...,1,0
2,15,spam,xxxmobilemovieclub use credit click wap link n...,1,0
4,19,spam,england v macedonia dont miss goal team news t...,1,0
14,47,ham,fair enough anything going,0,1
...,...,...,...,...,...
1633,5446,ham,back good journey let know need receipt shall ...,0,1
1634,5450,ham,sac need carry,0,1
1638,5457,ham,arun u transfr amt,0,1
1657,5524,spam,awarded sipix digital camera call 09061221061 ...,1,0
